In [1]:
import xgboost as xgb
import numpy as np
from sklearn.tree import DecisionTreeClassifier

1、xgBoost的基本使用  
2、自定义损失函数的梯度和二阶导  
3、binary:logistic/logitraw

In [7]:
def g_h(y_hat,y):
    p = 1.0 / (1.0 + np.exp(-y_hat))
    g = p - y.get_label()
    h = p * (1.0 - p)
    return g,h

In [8]:
def error_rate(y_hat,y):
    return 'error', float(sum(y.get_label() != (y_hat > 0.5))) / len(y_hat)

In [9]:
# 读取数据
data_train = xgb.DMatrix('agaricus_train.txt')
data_test = xgb.DMatrix('agaricus_test.txt')
print(data_train)
print(type(data_train))

<class 'xgboost.core.DMatrix'>


In [10]:
# 设置参数
param = {'max_depth': 3, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'} # logitraw
# param = {'max_depth': 3, 'eta': 0.3, 'silent': 1, 'objective': 'reg:logistic'}
watchlist = [(data_test, 'eval'), (data_train, 'train')]
n_round = 7
# bst = xgb.train(param, data_train, num_boost_round=n_round, evals=watchlist)
bst = xgb.train(param, data_train, num_boost_round=n_round, evals=watchlist, obj=g_h, feval=error_rate)

[0]	eval-error:0.016139	train-error:0.014433	eval-error:0.016139	train-error:0.014433
[1]	eval-error:0.016139	train-error:0.014433	eval-error:0.016139	train-error:0.014433
[2]	eval-error:0.016139	train-error:0.014433	eval-error:0.016139	train-error:0.014433
[3]	eval-error:0.016139	train-error:0.014433	eval-error:0.016139	train-error:0.014433
[4]	eval-error:0.002483	train-error:0.003071	eval-error:0.002483	train-error:0.003071
[5]	eval-error:0.002483	train-error:0.003071	eval-error:0.002483	train-error:0.003071
[6]	eval-error:0.002483	train-error:0.003071	eval-error:0.002483	train-error:0.003071


In [11]:
# 计算错误率
y_hat = bst.predict(data_test)
y = data_test.get_label()
print (y_hat)
print (y)
error = sum(y != (y_hat > 0.5))
error_rate = float(error) / len(y_hat)
print ('样本总数：\t', len(y_hat))
print ('错误数目：\t%4d' % error)
print ('错误率：\t%.5f%%' % (100*error_rate))

[  6.09937888e-06   9.84727502e-01   6.09937888e-06 ...,   9.99932647e-01
   4.45600620e-07   9.99932647e-01]
[ 0.  1.  0. ...,  1.  0.  1.]
样本总数：	 1611
错误数目：	   4
错误率：	0.24829%
